### 문제
- 공급처의 원재료 판매 가격 $p_i$, 공급 가능 가능일 $d_i$, 소비자의 구매 가격 $q_j$, 계약종료일 $e_j$ 이라 할 때, \
어떤 공급처와 소비자 회사 한쌍을 골라 계약을 맺고, 날짜 구간 $[d_i, e_j)$ 까지 물건을 한개씩 만든다고 할 때, 물건을 모두 팔았을 때의 최대 이익을 구하는 문제.
- 이익을 만들 수 없다면 0을 출력.

### 관찰
- 원재료가 $a$, 판매가 $b$, 계약 시작일 $s$, 계약 종료일 $e$ 라고 할 때, 해당 계약의 판매 이익은 $(b-a) \cdot (e-s)$ 이다.
- $N, M$ 이 50만이다. 적당히 $O(NM \log M)$ 정도의 시간복잡도를 가지는 알고리즘을 생각해야 한다.
- 좌표압축을 필요로 하는 값이 구성돼있다.

### 아이디어
- 우선 공급처와 판매처 모두 정렬 가능하다. $i, j$ 를 정렬된 list상에서의 공급처, 판매처의 인덱스라고 하자.
- $C[i][j] = i\text{ 번째 공급처에서 사고, } j \text{ 번째 소비자에게 팔 때 최대 이익 }$ 
  - (그리디)이때 같은 날짜의 공급처가 여러개 있으면, 공급 가격이 최소인 것을 고르고, 판매처가 여러개면 판매 가격이 최대인 것만 남겨서, $C$ 에서 $\max$를 고려하지 않아도 되게 한다.\
  즉, $i$ 번째 회사라는 것은, $cmp[i]$ 를 시작일로 가지는 회사중에서 가장 공급 가격이 싼 회사를 의미하고, $j$ 도 vice versa를 의미한다.
    - 좌표 압축시 추가 구현하면 될 것이다.
  - $C[i][j] = (e_j - d_i) \cdot  (q_j - p_i)$
  - 이렇게 해봤는데 $opt_c \leq opt_d$ 가 성립하지 않는다. 설령 그렇게 되도록 할 수 있다고 해도, 내가 아는 방법으로는 $O(N^2)$ 이하로 내릴 수 없었다.
- $DP[i][j] = i\text{ 번째 회사를 공급처로 정하고 } j \text{ 번째 소비자까지 고려했을 때 최대 이익 }$ 
- 지금보니 $O(NM \log N)$ 부터 문제가 있다. 둘다 50만인데 어떻게 풀것인가.

### 아이디어2
- 다른 접근이 필요한 것 같다. 만약 어떤 날짜에 대해서 각 회사마다 비용과 이익을 미리 알 수 있다면 어떨까?
  - 두 문제를 특정 날짜 $k$ 일에 대해서 일반화할 수 있어야 할 것이다.
  - 모든 공급처의 계약 일짜가 1일차부터 $k$ 일차라고 가정해보자. 즉, $d_i$ 가 높을수록 손해이도록 만드는 것이다. \
  $i$ 번째 구매처가 1일 부터 $k$ 일까지 계약한다고 했을 때 비용 $C(i, k)$ 은 다음과 같이 정의할 수 있다.\
  $C(i, k) = p_i \cdot \text{(계약 일수)} + (d-1) \cdot p_i = p_i \cdot (k - d_i) + (d - 1) \cdot p_i$
  - 비슷하게, $j$ 번째 판매처가 1일부터 $k$ 일까지 계약한다고 했을 때, $q_j$ 가 높을수록 이익이 높아지고, 초과 일자부터는 손해를 보도록 만들어보자\
   이익 $P(j, k)$ 는 다음과 같이 정의할 수 있다.\
    $P(j, k) = -|q_j \cdot (k - e_j + 1)| + (e_j - 1) \cdot q_j$
- 답이 안나온다 $opt_c \leq opt_d$ 를 도저히 보일 수 없음.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

INF = 4e18 + 7777
def sol() :
  M, N = map(int, input().split())
  P = [tuple(map(int, input().split())) for _ in range(M)]
  C = [tuple(map(int, input().split())) for _ in range(N)]
  P.sort()
  C.sort(reverse=True)

  A = []
  B = []
  for p in P :
    if not A or A[-1][1] > p[1] :
      A.append(p)
  
  for c in C :
    if not B or B[-1][1] < c[1] :
      B.append(c)
  B.reverse()
  
  S = [(0, len(A)-1, 0, len(B)-1)]
  ans = 0
  while S:
    s, e, l, r = S.pop()
    if s > e: continue

    m = (s + e) // 2
    res = -INF

    k = s
    for i in range(l, r+1) :
      dx = B[i][0] - A[m][0]
      dy = B[i][1] - A[m][1]
      area = 0 if dx < 0 and dy < 0 else dx * dy
      if area > res :
        res = area
        k = i

    S.append((s, m - 1, l, k))
    S.append((m + 1, e, k, r))
    ans = max(ans, res)
  print(ans)

sol()

### 풀이
- 기하학 풀이는 너무 어지럽다. $d_i < e_j$ 일때만 단조성을 띄는 것을 관찰하긴 했었는데, 그걸로 풀이를 할 생각을 안해봤다. 그것 먼저 해보자.